# Test Delight on DESC-DC2 simulation  in the context of  Vera C. Rubin Obs (LSST) 


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : January 22 2022



- run at NERSC with **desc-python** python kernel.


Instruction to have a **desc-python** environnement:
- https://confluence.slac.stanford.edu/display/LSSTDESC/Getting+Started+with+Anaconda+Python+at+NERSC


This environnement is a clone from the **desc-python** environnement where package required in requirements can be addded according the instructions here
- https://github.com/LSSTDESC/desc-python/wiki/Add-Packages-to-the-desc-python-environment

We will use the parameter file "tmps/parametersTestRail.cfg".
This contains a description of the bands and data to be used.
In this example we will generate mock data for the ugrizy LSST bands,
fit each object with our GP using ugi bands only and see how it predicts the rz bands.
This is an example for filling in/predicting missing bands in a fully bayesian way
with a flexible SED model quickly via our photo-z GP.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import sys,os
sys.path.append('../')
from delight.io import *
from delight.utils import *
from delight.photoz_gp import PhotozGP


#import logging
#import coloredlogs
#logger = logging.getLogger(__name__)
#coloredlogs.install(level='DEBUG', logger=logger,fmt='%(asctime)s,%(msecs)03d %(programname)s %(name)s[%(process)d] %(levelname)s %(message)s')


#print(sys.executable)
#print(sys.version)
#print(sys.version_info)

# Initialisation

In [2]:
workdir = "tmp"

# Configuration parameters

- now parameters are generated in a dictionnary

In [26]:
list_of_files = os.listdir(workdir)
list_of_files.remove('data') 
list_of_files.remove('delight_data') 
if '.ipynb_checkpoints' in list_of_files:
    list_of_files.remove('.ipynb_checkpoints')
list_of_configfiles = sorted(list_of_files)

In [27]:
list_of_configfiles

['parametersTest-Sens.cfg',
 'parametersTest-VC1e4_ell1e6.cfg',
 'parametersTest-chunks.cfg',
 'parametersTest.cfg',
 'parametersTest_1.cfg',
 'parametersTest_10.cfg',
 'parametersTest_11.cfg',
 'parametersTest_12.cfg',
 'parametersTest_13.cfg',
 'parametersTest_14.cfg',
 'parametersTest_15.cfg',
 'parametersTest_16.cfg',
 'parametersTest_17.cfg',
 'parametersTest_18.cfg',
 'parametersTest_19.cfg',
 'parametersTest_2.cfg',
 'parametersTest_20.cfg',
 'parametersTest_21.cfg',
 'parametersTest_21_VC1e4.cfg',
 'parametersTest_21_ellSig1e6.cfg',
 'parametersTest_21_zSig2.cfg',
 'parametersTest_3.cfg',
 'parametersTest_4.cfg',
 'parametersTest_5.cfg',
 'parametersTest_6.cfg',
 'parametersTest_7.cfg',
 'parametersTest_8.cfg',
 'parametersTest_9.cfg']

# Filters

- First, we must **fit the band filters with a gaussian mixture**. 
This is done with this script:

In [28]:
list_of_configfiles.remove('parametersTest-Sens.cfg')
list_of_configfiles.remove('parametersTest_21_ellSig1e6.cfg')
list_of_configfiles.remove('parametersTest-VC1e4_ell1e6.cfg')
list_of_configfiles.remove('parametersTest_21_zSig2.cfg')
list_of_configfiles.remove('parametersTest_21_VC1e4.cfg')
list_of_configfiles.remove('parametersTest-chunks.cfg')
#list_of_configfiles.remove('parametersTest.cfg')
list_of_configfiles
NCHUNKS = len(list_of_configfiles)

In [29]:
list_of_configfiles

['parametersTest.cfg',
 'parametersTest_1.cfg',
 'parametersTest_10.cfg',
 'parametersTest_11.cfg',
 'parametersTest_12.cfg',
 'parametersTest_13.cfg',
 'parametersTest_14.cfg',
 'parametersTest_15.cfg',
 'parametersTest_16.cfg',
 'parametersTest_17.cfg',
 'parametersTest_18.cfg',
 'parametersTest_19.cfg',
 'parametersTest_2.cfg',
 'parametersTest_20.cfg',
 'parametersTest_21.cfg',
 'parametersTest_3.cfg',
 'parametersTest_4.cfg',
 'parametersTest_5.cfg',
 'parametersTest_6.cfg',
 'parametersTest_7.cfg',
 'parametersTest_8.cfg',
 'parametersTest_9.cfg']

In [30]:
from delight.interfaces.rail.processFilters import processFilters

In [31]:
#configfilename = list_of_configfiles[9]
configfilename = 'parametersTest.cfg'
configfullfilename = os.path.join(workdir,configfilename) 
processFilters(configfullfilename)

2022-03-03 11:18:17,535 ipykernel_launcher.py delight.interfaces.rail.processFilters[120] INFO ----- processFilters ------
2022-03-03 11:18:17,538 ipykernel_launcher.py delight.interfaces.rail.processFilters[120] INFO parameter file is tmp/parametersTest.cfg


DC2LSST_u DC2LSST_g DC2LSST_r DC2LSST_i DC2LSST_z DC2LSST_y 

# SED

- Second, we will process the library of SEDs and project them onto the filters,
(for the mean fct of the GP) with the following script (which may take a few minutes depending on the settings you set):

In [32]:
from delight.interfaces.rail.processSEDs import processSEDs

In [33]:
#configfilename = list_of_configfiles[9]
configfilename = 'parametersTest.cfg'
configfullfilename = os.path.join(workdir,configfilename) 
processSEDs(configfullfilename)

2022-03-03 11:19:23,723 ipykernel_launcher.py, delight.interfaces.rail.processSEDs[120] INFO --- Process SED ---


# Train and apply
Run the scripts below. There should be a little bit of feedback as it is going through the lines.
For up to 1e4 objects it should only take a few minutes max, depending on the settings above.

## Template Fitting

In [34]:
from delight.interfaces.rail.templateFitting import templateFitting

In [35]:
for idx_file in range(1,NCHUNKS):
    theconfigfile = list_of_configfiles[idx_file]
    configfullfilename = os.path.join(workdir,theconfigfile) 
    templateFitting(configfullfilename)

2022-03-03 11:19:46,440 ipykernel_launcher.py, delight.interfaces.rail.templateFitting[120] INFO --- TEMPLATE FITTING ---
2022-03-03 11:19:46,444 ipykernel_launcher.py, delight.interfaces.rail.templateFitting[120] INFO ==> New Prior calculation from Benitez
2022-03-03 11:19:46,468 ipykernel_launcher.py, delight.interfaces.rail.templateFitting[120] INFO Thread number / number of threads: 1 , 1
2022-03-03 11:19:46,469 ipykernel_launcher.py, delight.interfaces.rail.templateFitting[120] INFO Input parameter file:tmp/parametersTest_1.cfg
2022-03-03 11:19:46,568 ipykernel_launcher.py, delight.interfaces.rail.templateFitting[120] INFO Number of Target Objects 507
2022-03-03 11:19:46,569 ipykernel_launcher.py, delight.interfaces.rail.templateFitting[120] INFO Thread 0 , analyzes lines 0 , to 507
2022-03-03 11:19:55,526 ipykernel_launcher.py, delight.interfaces.rail.templateFitting[120] INFO --- TEMPLATE FITTING ---
2022-03-03 11:19:55,527 ipykernel_launcher.py, delight.interfaces.rail.template

## Gaussian Process

### Training

In [36]:
from delight.interfaces.rail.delightLearn import delightLearn

In [37]:
delightLearn(configfullfilename)

2022-03-03 11:30:07,508 ipykernel_launcher.py, delight.interfaces.rail.delightLearn[120] INFO --- DELIGHT-LEARN ---
2022-03-03 11:30:07,664 ipykernel_launcher.py, delight.interfaces.rail.delightLearn[120] INFO Number of Training Objects 3755
2022-03-03 11:30:07,665 ipykernel_launcher.py, delight.interfaces.rail.delightLearn[120] INFO Thread 0 , analyzes lines 0 , to 3755


### Predictions

In [38]:
from delightApply_paramSpecPlot import delightApply_paramSpecPlot

In [ ]:
for idx_file in range(1,NCHUNKS):
    theconfigfile = list_of_configfiles[idx_file]
    configfullfilename = os.path.join(workdir,theconfigfile) 
    delightApply_paramSpecPlot(configfullfilename, plot=False, sensitivity=False)

2022-03-03 11:31:09,353 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO --- DELIGHT-APPLY ---
2022-03-03 11:31:09,497 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Number of Training Objects 3755
2022-03-03 11:31:09,498 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Number of Target Objects 507
2022-03-03 11:31:09,500 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Thread 0 , analyzes lines 0 to 507


Creation of GP with V_C = 0.1, V_L = 0.1, alpha_C = 1000.0, alpha_L = 100.0.
0 1.5922315120697021 0.21372175216674805 0.022856950759887695
100 1.3619506359100342 0.007479429244995117 0.016587018966674805
200 1.4453330039978027 0.007077693939208984 0.011363744735717773
300 1.3175082206726074 0.009799003601074219 0.03526139259338379
400 1.3937859535217285 0.007157564163208008 0.009971141815185547
500 1.357985258102417 0.01002645492553711 0.027922391891479492


2022-03-03 11:48:15,270 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO --- DELIGHT-APPLY ---
2022-03-03 11:48:15,394 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Number of Training Objects 3755
2022-03-03 11:48:15,395 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Number of Target Objects 361
2022-03-03 11:48:15,397 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Thread 0 , analyzes lines 0 to 361


Creation of GP with V_C = 0.1, V_L = 0.1, alpha_C = 1000.0, alpha_L = 100.0.
0 1.433784008026123 0.2128615379333496 0.011075496673583984
100 1.841963529586792 0.007154703140258789 0.009349584579467773
200 1.6180930137634277 0.009661197662353516 0.014428377151489258
300 1.3683655261993408 0.006735801696777344 0.012829065322875977


2022-03-03 12:02:47,770 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO --- DELIGHT-APPLY ---
2022-03-03 12:02:47,872 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Number of Training Objects 3755
2022-03-03 12:02:47,873 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Number of Target Objects 415
2022-03-03 12:02:47,874 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Thread 0 , analyzes lines 0 to 415


Creation of GP with V_C = 0.1, V_L = 0.1, alpha_C = 1000.0, alpha_L = 100.0.
0 1.4374268054962158 0.20329952239990234 0.023406505584716797
100 1.5101242065429688 0.008993387222290039 0.019840478897094727
200 1.4203197956085205 0.009018659591674805 0.02262091636657715
300 1.417372703552246 0.00846719741821289 0.01998615264892578
400 1.3070333003997803 0.007408618927001953 0.007936954498291016


2022-03-03 12:16:45,558 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO --- DELIGHT-APPLY ---
2022-03-03 12:16:45,721 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Number of Training Objects 3755
2022-03-03 12:16:45,722 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Number of Target Objects 448
2022-03-03 12:16:45,724 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Thread 0 , analyzes lines 0 to 448


Creation of GP with V_C = 0.1, V_L = 0.1, alpha_C = 1000.0, alpha_L = 100.0.
0 1.5962090492248535 0.18529367446899414 0.007731199264526367
100 1.3102612495422363 0.007312297821044922 0.008687496185302734
200 1.3821239471435547 0.006745576858520508 0.008297920227050781
300 1.4333915710449219 0.0066127777099609375 0.008762359619140625
400 1.6059815883636475 0.007334232330322266 0.008607864379882812


2022-03-03 12:31:49,893 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO --- DELIGHT-APPLY ---
2022-03-03 12:31:50,020 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Number of Training Objects 3755
2022-03-03 12:31:50,021 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Number of Target Objects 501
2022-03-03 12:31:50,022 ipykernel_launcher.py, delightApply_paramSpecPlot[120] INFO Thread 0 , analyzes lines 0 to 501


Creation of GP with V_C = 0.1, V_L = 0.1, alpha_C = 1000.0, alpha_L = 100.0.


# Analyze the outputs

In [ ]:
# First read a bunch of useful stuff from the parameter file.
params = parseParamFile(configfullfilename, verbose=False)
bandCoefAmplitudes, bandCoefPositions, bandCoefWidths, norms\
    = readBandCoefficients(params)
bandNames = params['bandNames']
numBands, numCoefs = bandCoefAmplitudes.shape
fluxredshifts = np.loadtxt(params['target_catFile'])
fluxredshifts_train = np.loadtxt(params['training_catFile'])
bandIndices, bandNames, bandColumns, bandVarColumns, redshiftColumn,\
            refBandColumn = readColumnPositions(params, prefix='target_')
redshiftDistGrid, redshiftGrid, redshiftGridGP = createGrids(params)
dir_seds = params['templates_directory']
dir_filters = params['bands_directory']
lambdaRef = params['lambdaRef']
sed_names = params['templates_names']
nt = len(sed_names)
f_mod = np.zeros((redshiftGrid.size, nt, len(params['bandNames'])))
for t, sed_name in enumerate(sed_names):
    f_mod[:, t, :] = np.loadtxt(dir_seds + '/' + sed_name + '_fluxredshiftmod.txt')

In [ ]:
# Load the PDF files
metrics = np.loadtxt(params['metricsFile'])
metricscww = np.loadtxt(params['metricsFileTemp'])
# Those of the indices of the true, mean, stdev, map, and map_std redshifts.
i_zt, i_zm, i_std_zm, i_zmap, i_std_zmap = 0, 1, 2, 3, 4
i_ze = i_zm
i_std_ze = i_std_zm

pdfs = np.loadtxt(params['redshiftpdfFile'])
pdfs_cww = np.loadtxt(params['redshiftpdfFileTemp'])
pdfatZ_cww = metricscww[:, 5] / pdfs_cww.max(axis=1)
pdfatZ = metrics[:, 5] / pdfs.max(axis=1)
nobj = pdfatZ.size
#pdfs /= pdfs.max(axis=1)[:, None]
#pdfs_cww /= pdfs_cww.max(axis=1)[:, None]
pdfs /= np.trapz(pdfs, x=redshiftGrid, axis=1)[:, None]
pdfs_cww /= np.trapz(pdfs_cww, x=redshiftGrid, axis=1)[:, None]

In [ ]:
ncol = 4
fig, axs = plt.subplots(5, ncol, figsize=(12, 12), sharex=True, sharey=False)
axs = axs.ravel()
z = fluxredshifts[:, redshiftColumn]
sel = np.random.choice(nobj, axs.size, replace=False)
lw = 2
for ik in range(axs.size):
    k = sel[ik]
    print(k, end=" ")
    axs[ik].plot(redshiftGrid, pdfs_cww[k, :],lw=lw, label='Standard template fitting')# c="#2ecc71", 
    axs[ik].plot(redshiftGrid, pdfs[k, :], lw=lw, label='New method')  #, c="#3498db"
    axs[ik].axvline(fluxredshifts[k, redshiftColumn], c="k", lw=1, label='Spec-z')
    ymax = np.max(np.concatenate((pdfs[k, :], pdfs_cww[k, :])))
    axs[ik].set_ylim([0, ymax*1.2])
    axs[ik].set_xlim([0, 3.1])
    axs[ik].set_yticks([])
    axs[ik].set_xticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4])
for i in range(ncol):
    axs[-i-1].set_xlabel('Redshift', fontsize=10)
axs[0].legend(ncol=3, frameon=False, loc='upper left', bbox_to_anchor=(0.0, 1.4))
#fig.tight_layout()
#fig.subplots_adjust(wspace=0.1, hspace=0.1, top=0.96)


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 12))
zmax = 3.1
rr = [[0, zmax], [0, zmax]]
nbins = 30
h = axs[0, 0].hist2d(metricscww[:, i_zt], metricscww[:, i_zm], nbins, cmap='Greys', range=rr)
hmin, hmax = np.min(h[0]), np.max(h[0])
axs[0, 0].set_title('CWW z mean')
axs[0, 1].hist2d(metricscww[:, i_zt], metricscww[:, i_zmap], nbins, cmap='Greys', range=rr, vmax=hmax)
axs[0, 1].set_title('CWW z map')
axs[1, 0].hist2d(metrics[:, i_zt], metrics[:, i_zm], nbins, cmap='Greys', range=rr, vmax=hmax)
axs[1, 0].set_title('GP z mean')
axs[1, 1].hist2d(metrics[:, i_zt], metrics[:, i_zmap], nbins, cmap='Greys', range=rr, vmax=hmax)
axs[1, 1].set_title('GP z map')
axs[0, 0].plot([0, zmax], [0, zmax], c='k')
axs[0, 1].plot([0, zmax], [0, zmax], c='k')
axs[1, 0].plot([0, zmax], [0, zmax], c='k')
axs[1, 1].plot([0, zmax], [0, zmax], c='k')
#fig.tight_layout()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(13, 6))
chi2s = ((metrics[:, i_zt] - metrics[:, i_ze])/metrics[:, i_std_ze])**2

axs[0].errorbar(metrics[:, i_zt], metrics[:, i_ze], yerr=metrics[:, i_std_ze], fmt='o', markersize=5, capsize=0)
axs[1].errorbar(metricscww[:, i_zt], metricscww[:, i_ze], yerr=metricscww[:, i_std_ze], fmt='o', markersize=5, capsize=0)
axs[0].plot([0, zmax], [0, zmax], 'k')
axs[1].plot([0, zmax], [0, zmax], 'k')
axs[0].set_xlim([0, zmax])
axs[1].set_xlim([0, zmax])
axs[0].set_ylim([0, zmax])
axs[1].set_ylim([0, zmax])
axs[0].set_title('New method')
axs[1].set_title('Standard template fitting')

fig.tight_layout()

In [ ]:
cmap = "coolwarm_r"
vmin = 0.0
alpha = 0.9
s = 5
fig, axs = plt.subplots(1, 2, figsize=(13, 5))
vs = axs[0].scatter(metricscww[:, i_zt], metricscww[:, i_zmap], 
                    s=s, c=pdfatZ_cww, cmap=cmap, linewidth=0, vmin=vmin, alpha=alpha)
vs = axs[1].scatter(metrics[:, i_zt], metrics[:, i_zmap], 
                    s=s, c=pdfatZ, cmap=cmap, linewidth=0, vmin=vmin, alpha=alpha)
clb = plt.colorbar(vs, ax=axs.ravel().tolist())
clb.set_label('Normalized probability at spec-$z$')
for i in range(2):
    axs[i].plot([0, zmax], [0, zmax], c='k', lw=1, zorder=0, alpha=1)
    axs[i].set_ylim([0, zmax])
    axs[i].set_xlim([0, zmax])
    axs[i].set_xlabel('Spec-$z$')
axs[0].set_ylabel('MAP photo-$z$')

axs[0].set_title('Standard template fitting')
axs[1].set_title('New method')

## Conclusion
Don't be too harsh with the results of the standard template fitting or the new methods since both have a lot of parameters which can be optimized!

If the results above made sense, i.e. the redshifts are reasonnable for both methods on the mock data, then you can start modifying the parameter files and creating catalog files containing actual data! I recommend using less than 20k galaxies for training, and 1000 or 10k galaxies for the delight-apply script at the moment. Future updates will address this issue.